In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter
import simpy
import random
import time 

In [3]:
class Location:
    def __init__(self, name=None, distance=None):
        self.name = name
        self.distance = distance

In [36]:
class Call:
    nb_call = 0 
    def __init__(self, ID=None, initial_time=None, final_time=None, status=None, location=None):
        Call.nb_call = Call.nb_call + 1
        self.ID = Call.nb_call
        self.initial_time = initial_time
        self.final_time = final_time
        self.status = status 
        self.location = location
         
        
    def detail(self):
        print('Call from {} received at {}'.format(self.location.name, self.initial_time))
        
    def generate_Call(self, env, beta, locations, distance, p, counter, boat_list, lastCall=None):
        self.location = Location()
        self.location.name = np.random.choice(locations, 1, p)
        self.location.distance = distance[' '.join(self.location.name)]

        if (lastCall==None):
            self.initial_time = float (np.random.exponential(scale=beta, size=1))
            yield env.timeout(self.initial_time)
        else:    
            self.initial_time = float (lastCall.initial_time + np.random.exponential(scale=beta, size=1))
            yield env.timeout(self.initial_time)

        print('%7.4f Call %d received from %s at a distance of %d' % (env.now, self.ID, self.location.name, self.location.distance))
        
        with counter.request() as req:
            yield req
            wait = env.now - self.initial_time #waiting time 
            print('%7.4f Call %d: Waited %6.3f' % (env.now, self.ID, wait))
            yield env.process(trajet(env, boat_list, self))
            print('%7.4f Call %d: Finished' % (env.now, self.ID))

        self.status = 1
        

In [37]:
def trajet(env, boat_list, call): 
    i = 0 
    while(i != len(boat_list) and boat_list[i].status != 0): #while you don't have a boat available 
        i = i + 1
    if(i != len(boat_list) and boat_list[i].status == 0): #if you have a boat available
        boat_list[i].status = 1 
        intervention = Intervention()
        print('Boat {} with a speed of {} respond to the call {} '.format(boat_list[i].ID, boat_list[i].speed, call.ID))
        intervention.boat = boat_list[i]
        intervention.call = call
        intervention.travel_time = call.location.distance/boat_list[i].speed
        intervention.TOS = 0.25
        intervention.response_time = 2*intervention.travel_time + intervention.TOS
        yield env.timeout(intervention.response_time)
        print('Duration of the intervention {} : {} '.format(intervention.ID, intervention.response_time))
        boat_list[i].status = 0
    

In [38]:
class Intervention: 
    nb_inter = 0 
    def __init__(self, travel_time=None, TOS=None, response_time=None, boat=None, call=None):
        Intervention.nb_inter = Intervention.nb_inter + 1 
        self.ID = Intervention.nb_inter
        self.travel_time = travel_time
        self.TOS = TOS
        self.response_time = response_time 
        self.boat = boat
        self.call = call

In [39]:
class Boat: 
    nb_boat = 0 
    def __init__(self, env, speed=None, status=None): 
        Boat.nb_boat = Boat.nb_boat + 1 
        self.ID = Boat.nb_boat 
        self.speed = speed 
        self.status = status 

In [40]:
def source(env, counter): 
    yield env.timeout(0)
    print('%7.4f Start' % (env.now))
    for i in range(5):
        call_list.append(Call())
        if i == 0: 
            env.process(call_list[i].generate_Call(env, 15,locations, distance, p, counter, boat_list))
        else: 
            env.process(call_list[i].generate_Call(env, 15, locations, distance, p, counter, boat_list, call_list[i-1]))

In [41]:
#Define the variables
locations = ['pooh', 'rabbit', 'piglet', 'Christopher']
distance = {"pooh":100, "rabbit":200, "piglet":300, "Christopher":400}
p=[0.5, 0.1, 0.1, 0.3]
nb_boat = 2

call_list = []
boat_list = []

In [42]:
#Setup and lauch of the simulation 
env = simpy.Environment()
boat_list.append(Boat(env,40,0))
boat_list.append(Boat(env,30,0))
counter = simpy.Resource(env, capacity=nb_boat)
env.process(source(env, counter))
env.run()

 0.0000 Debut de la simulation
 4.6156 Call 1 received from ['piglet'] at a distance of 300
 4.6156 Call 1: Waited  0.000
Boat 1 with a speed of 40 respond to the call 1 
 6.7856 Call 2 received from ['piglet'] at a distance of 300
 6.7856 Call 2: Waited  0.000
Boat 2 with a speed of 30 respond to the call 2 
10.3121 Call 3 received from ['pooh'] at a distance of 100
15.3309 Call 4 received from ['Christopher'] at a distance of 400
Duration of the intervention 1 : 15.25 
19.8656 Call 1: Finished
19.8656 Call 3: Waited  9.553
Boat 1 with a speed of 40 respond to the call 3 
Duration of the intervention 3 : 5.25 
25.1156 Call 3: Finished
25.1156 Call 4: Waited  9.785
Boat 1 with a speed of 40 respond to the call 4 
Duration of the intervention 2 : 20.25 
27.0356 Call 2: Finished
33.4476 Call 5 received from ['piglet'] at a distance of 300
33.4476 Call 5: Waited  0.000
Boat 2 with a speed of 30 respond to the call 5 
Duration of the intervention 4 : 20.25 
45.3656 Call 4: Finished
Duratio